# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
 # Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

#Access maps with unique API key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_weather_df = pd.read_csv("../Instructions/output_data/cities.csv")
city_weather_df


FileNotFoundError: [Errno 2] No such file or directory: '../Instructions/output_data/cities.csv'

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
#configure gmaps
gmaps.configure(api_key=g_key)

#Use the Lat and Lng as locations
locations = city_weather_df[["Lat","Lng"]]

#Add Heatmap layer to map
humidity = city_weather_df["Humidity"]
city_weather_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sosnogorsk,63.6023,53.8817,19.17,82,76,7.02,RU,1636865487
1,Mercedes,-34.6515,-59.4307,64.18,98,100,6.04,AR,1636865487
2,Itoman,26.1247,127.6694,73.78,73,75,19.57,JP,1636865471
3,Ushuaia,-54.8000,-68.3000,42.55,85,100,0.92,AR,1636865165
4,New Norfolk,-42.7826,147.0587,52.92,57,100,8.99,AU,1636865208


In [11]:
# map
figure = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights = humidity,
                                 dissipating=False, max_intensity=1,
                                 point_radius= 1)

# Add layer
figure.add_layer(heat_layer)
figure


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [12]:
smaller_city_df = city_weather_df.loc[(city_weather_df["Wind Speed"] <= 10) & (city_weather_df["Cloudiness"]== 0) & \
                  (city_weather_df["Max Temp"] >= 70) & (city_weather_df["Max Temp"] <= 80)].dropna()
smaller_city_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
104,Māndalgarh,25.2000,75.1000,74.70,28,0,4.68,IN,1636865557
217,Ixtapa,20.7000,-105.2000,70.84,78,0,2.91,MX,1636865642
264,Manoharpur,27.3000,75.9500,74.35,21,0,3.22,IN,1636865679
334,Coahuayana Viejo,18.7333,-103.6833,73.24,89,0,4.12,MX,1636865733
354,Yulara,-25.2406,130.9889,76.84,18,0,5.75,AU,1636865748
479,Rancho Palos Verdes,33.7445,-118.3870,78.67,53,0,1.01,US,1636865845
520,Bhag,29.0415,67.8239,78.33,8,0,3.56,PK,1636865878


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [13]:
#Store into variable named hotel_df
hotel_df = smaller_city_df.loc[:,["City","Country","Lat","Lng"]]

#Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Lat,Lng,Hotel Name
104,Māndalgarh,IN,25.2000,75.1000,
217,Ixtapa,MX,20.7000,-105.2000,
264,Manoharpur,IN,27.3000,75.9500,
334,Coahuayana Viejo,MX,18.7333,-103.6833,
354,Yulara,AU,-25.2406,130.9889,
479,Rancho Palos Verdes,US,33.7445,-118.3870,
520,Bhag,PK,29.0415,67.8239,


In [14]:
#Set parameters to search for hotels with 5000 meters.
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json?"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}


In [18]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
for index, row in hotel_df.iterrows():
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    params["location"] = f"{lat},{lng}"
    
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    #Save the hotel name to the dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
    except (KeyError, IndexError):
        print("Could not find result")
        print("-------------------------")
        #delay the code by one second to avoid an error
        time.sleep(1)
    print("--------------------Ending Search")

Retrieving Results for Index 70: ponta do sol.
Closest hotel in ponta do sol is Escarpas Resort.
--------------------Ending Search
Retrieving Results for Index 88: cidreira.
Closest hotel in cidreira is Hotel Castelo.
--------------------Ending Search
Retrieving Results for Index 138: morondava.
Closest hotel in morondava is Palissandre Cote Ouest resort & SPA.
--------------------Ending Search
Retrieving Results for Index 139: vaini.
Closest hotel in vaini is Dandeli Dreams Jungle Resort.
--------------------Ending Search
Retrieving Results for Index 176: nador.
Closest hotel in nador is Hôtel Mercure Rif Nador.
--------------------Ending Search
Retrieving Results for Index 297: mogok.
Closest hotel in mogok is MOGOK HILL HOTEL.
--------------------Ending Search
Retrieving Results for Index 444: birjand.
Closest hotel in birjand is هتل پرويز.
--------------------Ending Search


In [25]:

info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
# Add marker layer ontop of heat map


# Display figure


Figure(layout=FigureLayout(height='420px'))

In [26]:
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))